In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.insert(0,'../../modules')

In [2]:
import numpy as np
import factors

# Inference
In inference problems we want to figure out a distribution over a variable given other variables we do and do not know. That is what inference means. We want:

$$
\begin{aligned}
  P(Y|X_\text{known}) &= \sum_{X_\text{unknown}}P(Y,X_\text{unknown}|X_\text{known}) \\
\end{aligned}
$$
So we condition on known variables and then marginalize out the unknown variables we are not interested in. <br>
In Bayesian networks we can do exact inference with the factor representation.<br>
If we can represent a joint distribution with a factor, then conditioning and marginalizing are very simple, just using the formula above. Say you are interested in knowing whether you will need to get tea from the shop on the way home, given who is at home:

In [16]:
tea_factor = factors.Factor(["out of tea","mum","dad"],[2,2,2])
tea_factor.set([0,0,0],0.15)
tea_factor.set([0,0,1],0.05)
tea_factor.set([0,1,0],0.1)
tea_factor.set([0,1,1],0.35)
tea_factor.set([1,0,0],0.0)
tea_factor.set([1,0,1],0.1)
tea_factor.set([1,1,0],0.05)
tea_factor.set([1,1,1],0.2)
print(tea_factor)

out of tea  mum  dad  Values (10 dp)
0           0    0    0.15
0           0    1    0.05
0           1    0    0.1
0           1    1    0.35
1           0    0    0.0
1           0    1    0.1
1           1    0    0.05
1           1    1    0.2



In [31]:
#factors.condition(tea_factor,["out of tea"])

In [75]:
nope = factors.cut_variables(tea_factor,["dad"],[1])
print(nope)

out of tea  mum  Values (10 dp)
0           0    0.05
0           1    0.35
1           0    0.1
1           1    0.2

